# マイクロ波パルスで超伝導量子ビットを制御する

$\newcommand{\ket}[1]{|#1\rangle}$

In [ ]:
%pip install qudit-sim

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sciopt
import qutip as qtp
from qiskit import pulse, QuantumCircuit, IBMQ, transpile, schedule
from qiskit.circuit import Gate
from qiskit.providers.ibmq import least_busy

from qudit_sim import HamiltonianBuilder, pulse_sim
import qudit_sim.pulse as simpulse
import rqutils.paulis as paulis

sys.path.append('/home/jovyan/qc-workbook-lecturenotes')
from qc_workbook.bloch import draw_path
from qc_workbook.utils import operational_backend, find_best_chain

## 量子ビットのドライブシミュレーション

現実的なパラメータ値の量子ビットを定義し、$\ket{0}$を初期状態として一定振幅のドライブを加え、状態の変化を観察します。

In [ ]:
twopi = np.pi * 2.

# ωq = 2π×5GHz
qubit_frequency = 5.e+9 * twopi
# 今は量子ビット（最低2準位）のみを考えるので、非調和度（|1>↔︎|2>角振動数と|0>↔︎|1>角振動数の差）は実は関係ない
anharmonicity = -0.3e+9 * twopi
# ドライブの強さのスケール
drive_amplitude = 140.e+6 * twopi

hgen = HamiltonianBuilder(2)
# 量子ビットを定義
hgen.add_qudit(qubit_frequency=qubit_frequency, anharmonicity=anharmonicity, drive_amplitude=drive_amplitude, qudit_id='q0')
# ドライブを加える。角振動数はqubit_frequency, 振幅は上のdrive_amplitudeの0.4倍
hgen.add_drive('q0', frequency=qubit_frequency, amplitude=0.4)

In [ ]:
# シミュレーションでは時間を有限のステップで刻む。ここではドライブ1周期に対して10点取り、100周期シミュレートする
tlist = {'points_per_cycle': 10, 'num_cycles': 100}
# 初期状態は|0>
psi0 = qtp.basis(2, 0) # QuTiPという量子力学計算ライブラリにおける、2準位系の0基底
# X, Y, Zの期待値を各時刻で計算させる
e_ops = [qtp.sigmax(), qtp.sigmay(), qtp.sigmaz()]

# シミュレーションの実行
sim_result = pulse_sim(hgen, tlist, psi0=psi0, e_ops=e_ops)

`sim_result.times`にシミュレーションに使われた時刻の値が記録されている。

`sim_result.expect`は`e_ops`に対応して3つの要素を持つ配列で、各要素がX, Y, Zそれぞれの各時刻での期待値に対応する。

今は量子ビットを`qubit_frequency`で回転するフレームで観察しているので、状態はX軸周りに回転するはず。

In [ ]:
x, y, z = sim_result.expect
t = sim_result.times
draw_path(x, y, z, t)

$\langle Z \rangle$だけを時間に対してプロットする。

In [ ]:
plt.plot(t, z);

少しズームインすると、rotating-wave approximationで無視した高周波成分の影響も見える。

In [ ]:
plt.plot(t[:100], z[:100]);

X軸周りの回転なので、$\langle X \rangle$は時間変化しない。

In [ ]:
plt.plot(t, x, label=r'$\langle X \rangle$')
plt.plot(t, y, label=r'$\langle Y \rangle$')
plt.legend();

同じダイナミクスを回転しないフレームで観察する。

In [ ]:
hgen.set_global_frame('lab')
# 点の動きを見やすくするため、時刻を少しずらす
tlist = sim_result.times * 0.9
sim_result = pulse_sim(hgen, tlist, psi0=psi0, e_ops=e_ops)

x, y, z = sim_result.expect
t = sim_result.times
draw_path(x, y, z, t)

再び量子ビットのフレームに戻し、今度は位相オフセット$\pi/2$を加えたドライブをかける。

In [ ]:
hgen.set_global_frame('qudit')
hgen.clear_drive()
hgen.add_drive('q0', frequency=qubit_frequency, amplitude=(0.4 * np.exp(0.5j * np.pi)))

tlist = {'points_per_cycle': 10, 'num_cycles': 100}

sim_result = pulse_sim(hgen, tlist, psi0=psi0, e_ops=e_ops)

x, y, z = sim_result.expect
t = sim_result.times
draw_path(x, y, z, t)

実際の量子ビットでは高位順位があるので、操作はこんなに簡単ではない。

In [ ]:
# |0>, |1>, |2>を含める
hgen = HamiltonianBuilder(3)
# 量子ビットを定義
hgen.add_qudit(qubit_frequency=qubit_frequency, anharmonicity=anharmonicity, drive_amplitude=drive_amplitude, qudit_id='q0')
# ドライブを加える。角振動数はqubit_frequency, 振幅は上のdrive_amplitudeの0.4倍
hgen.add_drive('q0', frequency=qubit_frequency, amplitude=0.4)

In [ ]:
# シミュレーションでは時間を有限のステップで刻む。ここではドライブ1周期に対して10点取り、100周期シミュレートする
tlist = {'points_per_cycle': 10, 'num_cycles': 100}
# 初期状態は|0>
psi0 = qtp.basis(3, 0)
# X, Y, Zの期待値を各時刻で計算させる
sigmas = paulis.paulis(3)
e_ops = [qtp.Qobj(inpt=sigmas[1]), qtp.Qobj(inpt=sigmas[2]), qtp.Qobj(inpt=sigmas[3])]

# シミュレーションの実行
sim_result = pulse_sim(hgen, tlist, psi0=psi0, e_ops=e_ops)

In [ ]:
x, y, z = sim_result.expect
t = sim_result.times
draw_path(x, y, z, t)

In [ ]:
plt.plot(t, x, label=r'$\langle X \rangle$')
plt.plot(t, y, label=r'$\langle Y \rangle$')
plt.legend();

## 実機の量子ビットをX軸周りに回す

In [ ]:
# 認証し、バックエンドを選ぶ

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-utokyo', group='internal', project='qc-training22')

backend_list = provider.backends(filters=operational_backend(min_qubits=1))

# 今一番空いているバックエンド
backend = least_busy(backend_list)
print(backend.name())

In [ ]:
# 一番測定エラー率の低い量子ビットを選ぶ

config = backend.configuration()
prop = backend.properties()
defaults = backend.defaults()

best_qubit = 0
min_err = 1.
for iq in range(config.n_qubits):
    err = prop.readout_error(iq)
    if err < min_err:
        best_qubit = iq
        min_err = err
        
print(f'Using qubit {best_qubit}')

### Xゲートを調べる

In [ ]:
# 実際にXゲートとして使われているパルス列（Xなのでパルス一つ）
x_sched = defaults.instruction_schedule_map.get('x', best_qubit)

x_pulse = x_sched.instructions[0][1].pulse
print(x_pulse)

x_sched.draw()

これが具体的にどんな電気信号なのか、シミュレータで確認する。

In [ ]:
unit_time = 0.22e-9
drag = simpulse.Drag(duration=x_pulse.duration * unit_time,
                     amp=x_pulse.amp,
                     sigma=x_pulse.sigma * unit_time,
                     beta=x_pulse.beta * unit_time)

t = np.linspace(0., drag.duration, 3200)
signal = drag.modulate(1., defaults.qubit_freq_est[best_qubit] * twopi, 0., 0., 0., 'x', False)(t)

In [ ]:
plt.plot(t, signal.real)

これではよくわからないので、最初の200点だけを見る。

In [ ]:
plt.plot(t[:200], signal.real[:200])

### 実験

GaussianSquareパルスのプラトー長を変えて、それぞれのパルスで量子ビットがどのくらいブロッホ球上を回るか見る。

In [ ]:
# プラトーの幅
widths = np.arange(0, 640, 32)
# 振幅はXパルスの半分にしておく
amp = x_pulse.amp * 0.5
# ガウシアン部分のsigmaにはXパルスのものを使う
sigma = x_pulse.sigma

circuits = []

for width in widths:
    # パルスを定義
    duration = int(x_pulse.duration + width)
    gspulse = pulse.GaussianSquare(duration=duration, amp=amp, sigma=sigma, width=int(width))
    
    # パルススケジュールを組む
    with pulse.build(backend=backend) as sched:
        drive_chan = pulse.drive_channel(best_qubit)
        pulse.play(gspulse, drive_chan)
        
    circuit = QuantumCircuit(1)
        
    # 作ったスケジュールを新しいゲートとして回路に追加
    gate = Gate('gspulse', 1, [])
    circuit.add_calibration('gspulse', (best_qubit,), sched)
    circuit.append(gate, qargs=[0])
    
    circuit.measure_all()

    circuits.append(circuit)

circuits = transpile(circuits, backend=backend)

例として一つの回路の全体のパルススケジュールを確認する。

In [ ]:
schedule(circuits[10], backend=backend).draw()

In [ ]:
shots = 1000
job = backend.run(circuits, shots=shots)
counts = job.result().get_counts()

In [ ]:
z = np.array([c.get('0', 0) - c.get('1', 0) for c in counts]) / shots
plt.scatter(widths, z)

## Ramsey実験で共鳴周波数を求める

### 実験

In [ ]:
# SXパルス
sx_pulse = defaults.instruction_schedule_map.get('sx', best_qubit).instructions[0][1].pulse
sx_pulse

In [ ]:
circuits = []

# 予想される周波数から2MHzずらした周波数
frequency = round(defaults.qubit_freq_est[best_qubit] + 2.e+6, 6)

# SXパルス間の遅延を伸ばしていって、周波数のずれによる位相の進みを見る
delays = np.arange(512, 5120, 128)

for delay in delays:
    with pulse.build(backend=backend) as sched:
        drive_chan = pulse.drive_channel(best_qubit)
        # 周波数をずらす
        pulse.set_frequency(frequency, drive_chan)
        # SXパルスを打ち、delayだけ待って、再びSXを打つ
        pulse.play(sx_pulse, drive_chan)
        pulse.delay(int(delay), drive_chan)
        pulse.play(sx_pulse, drive_chan)
        
    gate = Gate('ramsey', 1, [])
    circuit = QuantumCircuit(1)
    circuit.add_calibration('ramsey', (best_qubit,), sched)

    circuit.append(gate, qargs=[0])
    circuit.measure_all()

    circuits.append(circuit)

circuits = transpile(circuits, backend=backend)

In [ ]:
# 一つ描いてみる
schedule(circuits[10], backend=backend).draw()

In [ ]:
shots = 1000
job = backend.run(circuits, shots=shots)
counts = job.result().get_counts()

In [ ]:
t = delays * unit_time
z = np.array([c.get('0', 0) - c.get('1', 0) for c in counts]) / shots
plt.scatter(t, z)

### 結果の解析（フィッティング）

In [ ]:
def curve(delay, amp, freq, off):
    return amp * np.cos(delay * freq * twopi + off)

popt, _ = sciopt.curve_fit(curve, t, z, p0=(1., 2.e+6, 0.))

plt.plot(t, curve(t, *popt))
plt.scatter(t, z)

In [ ]:
calibrated_frequency = 2.e+6 - popt[1] + defaults.qubit_freq_est[best_qubit]
print(f'Frequency update: {defaults.qubit_freq_est[best_qubit]} -> {calibrated_frequency}')